In [1]:
import os
os.chdir('../../' )
cwd=os.getcwd()

from oligo_designer_toolsuite.oligo_specificity_filter._filter_base import ProbeFilterBase
from oligo_designer_toolsuite.oligo_specificity_filter._filter_exact import ProbeFilterExact
from oligo_designer_toolsuite.oligo_specificity_filter._filter_blastn import ProbeFilterBlastn
from oligo_designer_toolsuite.oligo_specificity_filter._filter_bowtie import ProbeFilterBowtie

In [2]:
cwd

'/home/chels/CRISPR/repos/oligo-designer-toolsuite'

In [2]:
n_jobs=10
ligation_region=0
dir_output=cwd + "/output/"
dir_annotations=dir_output+"annotations/"
file_probe_info=dir_annotations + "oligo_DB_unknown_unknown_Custom_release_unknown_gene_transcript"
genes= ['WASH7P', 'DDX11L1', 'TRNT', 'NOC2L', 'PLEKHN1', 'AGRN','UBE2J2', 'DVL1', 'MIB2', 'LOC112268402_1' ]
min_probes_per_gene=2
file_transcriptome_fasta=dir_output +"annotations/reference_DB_unknown_unknown_Custom_release_unknown_genome_False_gene_transcript_True"
word_size=10
percent_identity=80
probe_length_min=30
probe_length_max=40
coverage=50


Test exact match filter

In [6]:
filter_exact=ProbeFilterExact(
        n_jobs,
        dir_output,
        genes,
        dir_annotations)

In [7]:
filter_exact.apply(probe_info)

INFO:filter_probes:Creating batches


TypeError: argument of type 'method' is not iterable

In [3]:
from Bio.SeqUtils import MeltingTemp as mt
Tm_parameters = {
        'check': True,
        'strict': True,
        'c_seq': None,
        'shift': 0,
        'nn_table': getattr(mt,'DNA_NN3'),
        'tmm_table': getattr(mt,'DNA_TMM1'),
        'imm_table': getattr(mt,'DNA_IMM1'),
        'de_table': getattr(mt, 'DNA_DE1'),
        'dnac1': 50, #[nM]
        'dnac2': 0,
        'selfcomp': False,
        'dNTPs': 0,
        'saltcorr': 7,
        'Na': 1.25, #[mM]
        'K': 75, #[mM]
        'Tris': 20, #[mM]
        'Mg': 10, #[mM] 
        }
Tm_correction_parameters = {
        'DMSO': 0,
        'DMSOfactor': 0.75,
        'fmdfactor': 0.65,
        'fmdmethod': 1,
        'GC': None,
        'fmd': 20}

In [4]:
os.chdir(cwd + "/oligo_designer_toolsuite")
from IO._database import CustomDB
from oligo_pre_filter._filter_base import MaskedSequences, GCContent, MeltingTemperature
from oligo_pre_filter._filter_padlock_probes import PadlockArms
annotation = dir_annotations+'GCF_000001405.40_GRCh38.p14_genomic.gtf'
sequence = dir_annotations+'GCF_000001405.40_GRCh38.p14_genomic.fna'
masked_sequences = MaskedSequences()
GC_content = GCContent(GC_content_min = 40, GC_content_max = 60)
melting_temperature = MeltingTemperature(Tm_min = 52, Tm_max= 67, Tm_parameters = Tm_parameters, Tm_correction_parameters = Tm_correction_parameters)
arms_tm = PadlockArms(min_arm_length=10,max_Tm_dif=2,Tm_min=40,Tm_max=43, Tm_parameters = Tm_parameters, Tm_correction_parameters = Tm_correction_parameters)
filters = [masked_sequences, GC_content, melting_temperature, arms_tm]
custom = CustomDB(probe_length_min = 30, probe_length_max=40, file_annotation=annotation, file_sequence=sequence, filters=filters)



/home/chels/CRISPR/repos/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:55: UserWarning: Species not specified.
  warnings.warn("Species not specified.")
/home/chels/CRISPR/repos/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:59: UserWarning: Genome assembly not specified.
  warnings.warn(f"Genome assembly not specified.")
/home/chels/CRISPR/repos/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:63: UserWarning: Annotation release not specified.
  warnings.warn(f"Annotation release not specified.")
/home/chels/CRISPR/repos/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:67: UserWarning: Annotation source not specified.
  warnings.warn(f"Annotation source not specified.")


In [15]:
custom.read_oligos_DB(file_probe_info)
probe_info=custom.oligos_DB

In [18]:
probe_info

{'WASH7P': {'ACCTACAAGATGGGGTACTAACACCACCCC': {'transcript_id': ['NR_024540.1',
    'NR_024540.1'],
   'exon_id': ['NR_024540.1_exon8', 'NR_024540.1_exon8_NR_024540.1_exon7'],
   'chromosome': '1',
   'start': ['16606', '16765'],
   'end': ['16636', '16795'],
   'strand': '-',
   'GC_content': '53.33',
   'melt_temp': '60.16',
   'melt_temp_arm1': '41.64',
   'melt_temp_arm2': '41.55',
   'dif_melt_temp_arms': '0.09',
   'ligation_site': '15'},
  'CCTACAAGATGGGGTACTAACACCACCCCC': {'transcript_id': ['NR_024540.1',
    'NR_024540.1'],
   'exon_id': ['NR_024540.1_exon8', 'NR_024540.1_exon8_NR_024540.1_exon7'],
   'chromosome': '1',
   'start': ['16607', '16766'],
   'end': ['16637', '16796'],
   'strand': '-',
   'GC_content': '56.67',
   'melt_temp': '60.6',
   'melt_temp_arm1': '41.67',
   'melt_temp_arm2': '42.73',
   'dif_melt_temp_arms': '1.06',
   'ligation_site': '16'},
  'CTACAAGATGGGGTACTAACACCACCCCCA': {'transcript_id': ['NR_024540.1',
    'NR_024540.1'],
   'exon_id': ['NR_0245

In [16]:
import pandas as pd
df = pd.DataFrame.from_dict(probe_info)

Test Blast filter

In [6]:
blast_filter=ProbeFilterBlastn(n_jobs,
        file_transcriptome_fasta, 
        dir_output,
        genes, word_size,
        percent_identity, coverage, probe_length_min, probe_length_max, ligation_region)

In [7]:
blast_filter.apply(probe_info)

INFO:filter_probes:Creating batches


TypeError: argument of type 'method' is not iterable

Test Bowtie filter

In [5]:
n_jobs=10
min_mismatches=4
mismatch_region=5

bowtie_filter=ProbeFilterBowtie(n_jobs,
        dir_output,
        file_probe_info,
        genes,
        probe_length_min,
        probe_length_max,
        file_transcriptome_fasta, min_mismatches, mismatch_region)



In [6]:
bowtie_filter.apply()



# reads processed: 339
# reads with at least one reported alignment: 339 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 501 alignments to 1 output stream(s)
# reads processed: 24
# reads with at least one reported alignment: 24 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 24 alignments to 1 output stream(s)
# reads processed: 426
# reads with at least one reported alignment: 426 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 466 alignments to 1 output stream(s)
# reads processed: 546
# reads with at least one reported alignment: 546 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 1046 alignments to 1 output stream(s)
# reads processed: 294
# reads with at least one reported alignment: 294 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 337 alignments to 1 output stream(s)
# reads processed: 307
# reads with at least one reported alignment: 307 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 2967 alignments to 1 outp

INFO:filter_probes:Bowtie alignmnet search finished in 117.55269490000137 seconds


# reads processed: 1236
# reads with at least one reported alignment: 1236 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 3735 alignments to 1 output stream(s)


INFO:filter_probes:Bowtie results processed in 0.47547449999910896 seconds
